In [2]:
import os

# Path to the directory containing the labels
labels_directory = 'E:\datasets'

# Iterate over the labels subdirectories
for root, dirs, files in os.walk(labels_directory):
    for file in files:
        if file.endswith('.txt'):
            file_path = os.path.join(root, file)
            with open(file_path, 'r') as f:
                lines = f.readlines()

            # Modify the YOLO bounding box coordinates in each line
            modified_lines = []
            for line in lines:
                values = line.strip().split(' ')
                class_label = values[0]
                modified_values = [class_label]

                for i in range(1, len(values)):
                    value = float(values[i])
                    if i % 2 == 1:  # X-coordinate
                        if value < 0:
                            value = 0
                        elif value > 1:
                            value = 1
                    modified_values.append(str(value))

                modified_line = ' '.join(modified_values) + '\n'
                modified_lines.append(modified_line)

            # Write the modified lines back to the file
            with open(file_path, 'w') as f:
                f.writelines(modified_lines)
